<a href="https://colab.research.google.com/github/Richish/hands_on_ml/blob/master/6_decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training and visualizing a decision tree

## Training for iris dataset classification

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X= iris.data[:,2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)



DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [3]:
# visualizing
from sklearn.tree import export_graphviz
export_graphviz(
    decision_tree=tree_clf, out_file="tree_visual.dot", feature_names=iris.feature_names[2:], class_names=iris.target_names, rounded=True, filled=True
)

In [6]:
!ls
#!dot -Tpng tree_visual.dot -o iris_tree.png

sample_data  somefile.png  tree_visual.dot


In [7]:
#!dot -Tpng tree_visual.dot -o iris_tree.png
import pydot

(graph,) = pydot.graph_from_dot_file('tree_visual.dot')
graph.write_png('tree_visual.png')

## Note - Training for decision tree doesn't require data scaling or centering.

## Attributes of a node:


1.   Samples: How many sample does the current node have/process.
2.   Values: List of fixed length(=no. of classes). ith item in list = how many samples in current node belong to ith class.
3.   Gini (only for leaf node): Measure of 'impurity' of a node. Lower(=0) means more pure. If current leaf node has samples that all belong to a single class then that node is called pure. More distributed the samples more impure the node becomes.

If a node has values = [10,20,30], impurity score = 1 - (10/60)^2 - (20/60)^2 -(30/60)^2

If a node has values = [60,0,0] or [0,60,0] or [0,0,60], then purity = 1 - (60/60)^2 - (0/60)^2 - (0/60)^2 = 0
 
Gini impurity of ith node = G(i) = 1 − (k= 1 to n)Σ (p(i, k))^2
Here p(i,k) = The ratio of class k instances among the training instances in the ith node



## Sklearn only has 2 children of their nodes.
Scikit-Learn uses the CART algorithm, which produces only binary
trees: nonleaf nodes always have two children (i.e., questions only
have yes/no answers). However, other algorithms such as ID3 can
produce Decision Trees with nodes that have more than two children.

# Estimating Class Probabilities

A Decision Tree can also estimate the probability that an instance belongs to a particular
class k: first it traverses the tree to find the leaf node for this instance, and then it
returns the ratio of training instances of class k in this node. For example, suppose
you have found a flower whose petals are 5 cm long and 1.5 cm wide. The corresponding
leaf node is the depth-2 left node, so the Decision Tree should output the
following probabilities: 0% for Iris-Setosa (0/54), 90.7% for Iris-Versicolor (49/54),
and 9.3% for Iris-Virginica (5/54). And of course if you ask it to predict the class, it
should output Iris-Versicolor (class 1) since it has the highest probability.

In [9]:
tree_clf.predict_proba([[5, 1.5]]) # predict probabilities

array([[0.        , 0.90740741, 0.09259259]])

In [10]:
tree_clf.predict([[5, 1.5]]) # classification only

array([1])

## Notice that the estimated probabilities would be exactly same for all the samples that end up inside same leaf node.

#CART(Classification and Regression Tree) Training Algorithm
CART- Classification and Regression Tree

First splits the training set in two subsets using a single feature k and a threshold t(subscript k) (e.g., “petal length ≤ 2.45 cm”). How does it choose k and t(subscript k)? It searches for the
pair (k, t(subscript k)) that produces the purest subsets (weighted by their size). The cost function that the algorithm tries to minimize is given as:

J(k, tk) = (m(left)/m)*G(left) + (m(right)/m)*G(right)

Here, 

m = total no. of samples in this node.
m(left/right) = no. of samples that go to left/right child.
G(left/right) = impurity of left/right child node.

Once it has successfully split the training set in two, it splits the subsets using the same logic, then the sub-subsets and so on, recursively. It stops recursing once it reaches the maximum depth (defined by the max_depth hyperparameter), or if it cannot find a split that will reduce impurity. A few other hyperparameters control additional stopping conditions (min_samples_split, min_sam
ples_leaf, min_weight_fraction_leaf, and max_leaf_nodes).

## Cart is a greedy algorithm
CART algorithm is a greedy algorithm: it greedily
searches for an optimum split at the top level, then repeats the
process at each level. It does not check whether or not the split will
lead to the lowest possible impurity several levels down. 
A greedy
algorithm often produces a reasonably good solution, but it is not
guaranteed to be the optimal solution.

1. Finding the optimal tree is known to be an NPComplete
problem.
2. It requires O(exp(m)) time, making the problem
intractable even for fairly small training sets. This is why we
must settle for a “reasonably good” solution.

## Computational complexity of CART

1. Prediction(not training): Decision Trees are generally approximately balanced, so traversing the Decision Tree requires going through roughly O(log2(m)) nodes. Since each node only requires checking the value of one feature, the overall prediction complexity is just O(log2(m)), independent of the number of features. So predictions are very fast, even when dealing with large training sets.

2. Training: The training algorithm compares all features (or less if max_features is set)
on all samples at each node. This results in a training complexity of O(n × m log(m)). For small training sets (less than a few thousand instances), Scikit-Learn can speed up training by presorting the data (set presort=True), but this slows down training considerably for larger training sets.

# Entropy instead of Gini impurity can be used

By default, the Gini impurity measure is used, but you can select the entropy impurity measure instead by setting the criterion hyperparameter to "entropy".

A set’s entropy is zero when it contains instances of only one class. 

Entropy, H{i} = − (for (k = 1 to n) and p{i,k}!=0)Σ(p{i,k}).log(p{i,k})

note- values in {} represent subscript here.


## To use Entropy or Impurity ?

Most of the time it does not make a big difference: they lead to similar trees. 
1. Gini impurity is slightly faster to compute, so it is a good default. 
2. Entropy tends to produce slightly more balanced trees. Gini impurity tends to
isolate the most frequent class in its own branch of the tree(But this happens only rarely so impurity can be the default).

# Regularization hyperparameters

Decision Trees make very few assumptions about the training data (as opposed to linear
models, which obviously assume that the data is linear, for example). If left
unconstrained, the tree structure will adapt itself to the training data, fitting it very
closely, and most likely overfitting it. Such a model is often called a nonparametric
model, not because it does not have any parameters (it often has a lot) but because the
number of parameters is not determined prior to training, so the model structure is
free to stick closely to the data. In contrast, a parametric model such as a linear model
has a predetermined number of parameters, so its degree of freedom is limited,
reducing the risk of overfitting (but increasing the risk of underfitting).

In Scikit-Learn, regularization is done by
max_depth hyperparameter (the default value is None, which means unlimited).
Reducing max_depth will regularize the model and thus reduce the risk of overfitting.

Other parameters: 
1. min_samples_split (the minimum number of samples
a node must have before it can be split)
2. min_samples_leaf (the minimum number
of samples a leaf node must have)
3. min_weight_fraction_leaf (same as
min_samples_leaf but expressed as a fraction of the total number of weighted instances)
4. max_leaf_nodes (maximum number of leaf nodes)
5. max_features (maximum number of features that are evaluated for splitting at each node).

Increasing
min_* hyperparameters or reducing max_* hyperparameters will regularize the
model.

## Other not so common way:
First training the Decision Tree without
restrictions, then pruning (deleting) unnecessary nodes. 
A node whose children are all leaf nodes is considered unnecessary if the
purity improvement it provides is not statistically significant. 
Standard statistical tests, such as the χ2 test, are used to estimate the
probability that the improvement is purely the result of chance. If this probability, called the pvalue,
is higher than a given threshold (typically 5%, controlled by
a hyperparameter), then the node is considered unnecessary and its
children are deleted. The pruning continues until all unnecessary
nodes have been pruned.

# Regression



In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

This tree looks very similar to the classification tree you built earlier. The main differencesa are:
1. Instead of predicting a class in each node, it predicts a value(and not a values list).
2. This prediction is simply the average target value of training
instances associated to this node. 
3. This prediction results in a Mean Squared Error associated to that node, this is MSE for the samples in that node.

The algorithm splits sample in each node(to be passed on to child nodes) in a way that makes most training instances as close as possible to that predicted value.

## CART cost function for regression
Uses MSE instead of gini to calculate cost. Otherwise, it is same as that for classification.

{} represent subscript in following eqns.

J(k, t{k}) = (m{left}/m)*MSE(left) + (m{right}/m)*MSE{right}

here, MSE{node} = (for i in all nodes) Σ ( ÿ{node} - y^(i}) )^2
and, ÿ{node} = (1/m{node}) * (for i in node) Σ y^(i)

# Instability
## Decision Trees love orthogonal decision boundaries (all splits are perpendicular to an axis).

So even in case original dataset is easily linearly seperable, just rotating the axis of features(say 45 degree)- results in very complex decision tree since decision boundaries are orthogonal and rotated fatures will require multiple of orthogonal boundaries to work.

Hence training set that may be easily linearly seperable may require uneccesarily complex models that may not perform well.

One way to limit this problem is to use PCA, which often results in a better orientation of the training data.

More generally, the main issue with Decision Trees is that they are 
## very sensitive to small variations in the training data. 
For example, if you just remove the widest Iris-
Versicolor from the iris training set (the one with petals 4.8 cm long and 1.8 cm wide) and train a new Decision Tree, you may get the model that  looks very different from the previous Decision Tree.

Also, since the training algorithm used by Scikit-Learn is stochastic you may
get very different models even on the same training data (unless you set the
random_state hyperparameter).



### Random forests are a solution to instability.
Random Forests can limit this instability by averaging predictions over many trees.